# Transfer learning met PyTorch

In deze notebook gaan we het concept van transfer learning onderzoeken. Transfer learning is een techniek waarbij een pre-trained neuraal netwerk wordt gebruikt als startpunt voor een nieuwe taak. Er zijn twee belangrijke benaderingen binnen transfer learning:

1. **Feature Extractie**: Hierbij houden we de gewichten van het pre-trained model vast en gebruiken we het model als feature extractor. Alleen het laatste (geclassificeerde) deel van het netwerk wordt aangepast aan de nieuwe taak.
  
2. **Fine-Tuning**: Hierbij worden de gewichten van (een deel van) het pre-trained model verder getraind op de nieuwe taak. Dit maakt het mogelijk om het model verder aan te passen aan de specifieke data.

## Stappen in de notebook

1. **Data voorbereiding**:
   - We gebruiken een kleine dataset voor classificatie (bijvoorbeeld een subset van de CIFAR-10 dataset).

2. **Model laden en voorbereiden**:
   - We laden een pre-trained model (bijv. ResNet-18) uit PyTorch's `torchvision` bibliotheek.
   - We maken aanpassingen voor beide benaderingen van transfer learning.

3. **Training**:
   - We demonstreren zowel de **feature extractie** als de **fine-tuning** methode.
   - We trainen de laatste laag voor de feature extractie en een deel van het netwerk voor fine-tuning.

4. **Evaluatie**:
   - We evalueren de prestaties van beide benaderingen op een testset.

---

## 1. Data Voorbereiding

We beginnen met het importeren van de benodigde bibliotheken en het laden van de dataset.


In [2]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 22.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 15.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 23.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import random

# Data transformaties
transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 pixels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Laden van de CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
total_samples = len(trainset)
sample_size = int(0.01 * total_samples)
random_indices = random.sample(range(total_samples), sample_size)
trainset_reduced = Subset(trainset, random_indices)
trainloader = DataLoader(trainset_reduced, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
total_samples = len(testset)
sample_size = int(0.01 * total_samples)
random_indices = random.sample(range(total_samples), sample_size)
testset_reduced = Subset(testset, random_indices)

# Create a DataLoader for the reduced dataset
testloader = DataLoader(testset_reduced, batch_size=32, shuffle=True, num_workers=2)

# Klassen in CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:12<00:00, 14079767.24it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## 2. Model Laden en Voorbereiden

### A) Feature Extractie

In deze aanpak houden we de gewichten van het pre-trained model vast en passen we alleen de laatste laag aan.

In [3]:
import torchvision.models as models

model_fe = models.resnet18(pretrained=True)

print(model_fe)

for param in model_fe.parameters():
    param.requires_grad=False # deze parameter moet niet getrained worden

num_ftrs = model_fe.fc.in_features
model_fe.fc = nn.Linear(num_ftrs, 10) # vervang de fc blok door 1 lineaire laag met 10 outputs

c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jens.baetens3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### B) Fine-Tuning

In deze aanpak staan we toe dat sommige (of alle) gewichten verder worden getraind.

In [ ]:
import torchvision.models as models

model_ft = models.resnet18(pretrained=True)

print(model_ft)

for name, param in model_ft.named_parameters():
    if "layer4" in name:
        break
    else:
        param.requires_grad=False # deze parameter moet niet getrained worden

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10) # vervang de fc blok door 1 lineaire laag met 10 outputs

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 3. Training

Hier zullen we beide modellen trainen, eerst het model voor feature extractie, gevolgd door het model voor fine-tuning.

### A) Feature Extractie Training

In [5]:
import time

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_fe.parameters(), lr=0.01)

model_fe.train()
for epoch in range(5):
    running_loss = 0.0
    start_time = time.time()

    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model_fe(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    end_time = time.time()

    print(f"Epoch {epoch}, trained in {end_time- start_time} with loss {running_loss/len(trainloader)}")

Epoch 0, trained in -17.74085545539856 with loss 3.0900109484791756
Epoch 1, trained in -16.910815238952637 with loss 1.260518878698349
Epoch 2, trained in -16.38392472267151 with loss 0.7451103664934635
Epoch 3, trained in -16.666611671447754 with loss 0.6270784959197044
Epoch 4, trained in -16.239169359207153 with loss 0.5284742601215839


### B) Fine-Tuning Training

In [6]:
import time

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr=0.01)

model_ft.train()
for epoch in range(5):
    running_loss = 0.0
    start_time = time.time()

    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    end_time = time.time()

    print(f"Epoch {epoch}, trained in {start_time-end_time} with loss {running_loss/len(trainloader)}")

Epoch 0, trained in -18.859859704971313 with loss 2.787108711898327
Epoch 1, trained in -18.667527437210083 with loss 1.3967593237757683
Epoch 2, trained in -19.380707263946533 with loss 1.1951563693583012
Epoch 3, trained in -21.353277921676636 with loss 0.7864222060889006
Epoch 4, trained in -20.574011087417603 with loss 0.5929177459329367


## 4. Evaluatie
Na het trainen evalueren we de prestaties van beide modellen op de testset.

### A) Evaluatie van het Feature Extractie Model

In [ ]:
correct = 0
total = 0

model_fe.eval()
for inputs, labels in testloader:
    # zero_grad niet nodig in eval modus
    predictions = model_fe(inputs) # kansen uit (10 kansen per input)
    _, predicted = torch.max(predictions, 1)
    
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f"Accuracy is {correct/total}")

torch.Size([32, 3, 224, 224])
torch.Size([32, 10])
torch.Size([32, 3, 224, 224])
torch.Size([32, 10])
torch.Size([32, 3, 224, 224])
torch.Size([32, 10])
torch.Size([4, 3, 224, 224])
torch.Size([4, 10])
Accuracy is 0.71


### B) Evaluatie van het Fine-Tuning Model

In [ ]:
correct = 0
total = 0

model_ft.eval()
for inputs, labels in testloader:
    # zero_grad niet nodig in eval modus
    predictions = model_ft(inputs) # kansen uit (10 kansen per input)
    _, predicted = torch.max(predictions)
    
    total += labels.size()
    correct += (predicted == labels).sum().item()

print(f"Accuracy is {correct/total}")

## Oefening: transfer learning met extra lagen voor FashionMNIST

### Oefeningomschrijving:
In deze oefening ga je een neuraal netwerk trainen met de FashionMNIST dataset, gebruikmakend van transfer learning. Je zult een pre-trained ResNet-18 model gebruiken, en daar drie extra volledig verbonden lagen aan toevoegen. Vervolgens train je het model en evalueer je de prestaties.

### Stappen:

**Data Voorbereiding:**

Laad de FashionMNIST dataset en breng de nodige transformaties aan.
Splits de data in een trainingsset en een testset.

**Model Voorbereiding:**

Laad een pre-trained ResNet-18 model.
Pas het model aan door drie extra volledig verbonden lagen toe te voegen:
* De eerste extra laag moet 512 neuronen hebben en een ReLU activatiefunctie.
* De tweede extra laag moet 256 neuronen hebben en een ReLU activatiefunctie.
* De derde extra laag moet 128 neuronen hebben en een ReLU activatiefunctie.
* De laatste output laag moet het aantal klassen in FashionMNIST (10 klassen) bevatten.

**Training:**

Definieer een loss-functie en een optimizer.
Train het model voor een aantal epochs.
Meet en print de tijd die elke epoch kost.

**Evaluatie:**

Evalueer de prestaties van het getrainde model op de testset en rapporteer de accuraatheid.

**Vragen om te beantwoorden:**

* Wat is het effect van de extra lagen op de prestaties van het model?
* Hoeveel tijd kost elke epoch en hoe varieert dit met het aantal lagen?